In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.utils.prune as prune
import torchvision.models as models
import torchvision.datasets as datasets
import time
import test
import train


C:\Users\ashis\AppData\Roaming\Python\Python39\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\ashis\AppData\Roaming\Python\Python39\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
##Dataset - CIFAR10 Dataset for demonstration

trn_batch_size = 64
val_batch_size = 64


train_tfms = transforms.Compose([
transforms.RandomCrop(32, padding = 4),
transforms.RandomHorizontalFlip(),
transforms.ToTensor(),
transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

valid_tfms = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])


fullset = datasets.CIFAR10(root='./data10', train=True, download=True, transform=train_tfms)
testset = datasets.CIFAR10(root='./data10', train=False, download=True, transform=valid_tfms)

trainloader = torch.utils.data.DataLoader(fullset, batch_size=trn_batch_size,
                                            shuffle=False, pin_memory=True, num_workers=1)

valloader = torch.utils.data.DataLoader(testset, batch_size=val_batch_size,
                                        shuffle=False, pin_memory=True, num_workers=1)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Model ResNet18 for Demonstration purpose

device = "cuda" if torch.cuda.is_available() else "cpu"

model = models.resnet50().to(device)
model.load_state_dict(torch.load('resnet50.pth'))

<All keys matched successfully>

In [5]:

criterion = nn.CrossEntropyLoss()
# Evaluate the model using the imported module
start = time.time()
test.evaluate_model(
    model=model,
    test_loader=trainloader,
    criterion=criterion,
    device=device
)
print(f"Inference time Before Pruning: {time.time() - start}s")

Test Loss: 25.2740, Test Accuracy: 0.02%
Inference time Before Pruning: 56.046891927719116s


In [6]:
#Single layer Pruning

prune.l1_unstructured(model.conv1, name="weight", amount=0.25)

# Check the pruned weights and the mask
print("Pruned weights for conv1:\n", model.conv1.weight)
print("Mask applied to conv1 weights:\n", model.conv1.weight_mask)

Pruned weights for conv1:
 tensor([[[[-0.0000, -0.0000,  0.0374,  ...,  0.0479, -0.0000,  0.0000],
          [-0.0574,  0.0447,  0.0775,  ...,  0.0884,  0.0293, -0.0583],
          [ 0.0684, -0.2704,  0.4035,  ..., -0.1649,  0.2187, -0.0729],
          ...,
          [-0.1087,  0.3815, -0.4549,  ...,  0.6837, -0.5786,  0.2246],
          [ 0.0257, -0.1770,  0.6437,  ...,  0.5264, -0.0493, -0.0681],
          [ 0.0453, -0.1307,  0.0000,  ..., -0.3575,  0.1898, -0.0000]],

         [[ 0.0000,  0.0000, -0.0000,  ...,  0.0869, -0.0655,  0.0000],
          [-0.0335,  0.0391,  0.0866,  ...,  0.1171,  0.0000, -0.0441],
          [ 0.0553, -0.2642,  0.4269,  ..., -0.2372,  0.3202, -0.1183],
          ...,
          [-0.1266,  0.5317, -0.6581,  ...,  0.9423, -0.7809,  0.2364],
          [-0.0262, -0.1331,  0.7207,  ...,  0.6755, -0.0447, -0.0769],
          [ 0.0575, -0.1175,  0.0612,  ..., -0.4092,  0.2330, -0.0529]],

         [[-0.0000,  0.0000, -0.0381,  ...,  0.0399, -0.0556,  0.0358],
   

In [7]:
# Pruning Multiple layers
layers_to_prune = ["conv1", "layer1.0.conv1"]

for layer_name in layers_to_prune:
   
    modules = layer_name.split('.')
    layer = model
    for module in modules:
        if module.isdigit():
            layer = layer[int(module)] 
        else:
            layer = getattr(layer, module)

    prune.l1_unstructured(layer, name="weight", amount=0.25)


In [8]:
# Calculate Inference time after pruning
criterion = nn.CrossEntropyLoss()

start = time.time()
test.evaluate_model(
    model=model,
    test_loader=trainloader,
    criterion=criterion,
    device=device
)
print(f"Inference time After Pruning: {time.time() - start}s")

Test Loss: 25.5385, Test Accuracy: 0.01%
Inference time After Pruning: 61.08184218406677s


In [ ]:
# Finetune for 10 Epochs

batch_size = 64
num_epochs = 10
optimizer_name = 'Adam'
scheduler_name = 'StepLR'
lr = 0.001
step_size = 10
gamma = 0.1
model_path = 'pmodel.pth'

# Train and save the model using the imported module
saved_model_path = train.train_and_save_model(
    model=model,
    train_loader=trainloader,
    num_epochs=num_epochs,
    optimizer_name=optimizer_name,
    scheduler_name=scheduler_name,
    lr=lr,
    step_size=step_size,
    gamma=gamma,
    model_path=model_path,
    mask_dict=None
)

print(f"Trained model saved to {saved_model_path}")


Epoch [1/10], Loss: 1.2977, Accuracy: 63.84%
Epoch [2/10], Loss: 0.6320, Accuracy: 79.05%
